In [1]:
import os
os.environ["KERAS_BACKEND"] = 'tensorflow'

In [2]:
from keras_core import layers
from keras_core import ops
import tensorflow as tf
from keras_core import models
from modules import FourierConv2D, FourierCNN
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

Using TensorFlow backend


2023-10-06 15:32:09.686260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 15:32:10.436247: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model = FourierCNN()
model(tf.zeros((1, 224, 224, 3)))
model.summary()

2023-10-06 15:32:12.516672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-06 15:32:12.535012: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-06 15:32:12.535067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-06 15:32:12.536462: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-06 15:32:12.536544: I tensorflow/compile

Model: "fourier_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ fourier_cnn/stem (Sequential)   │ ?                         │      4,800 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/downsampling_0      │ ?                         │     37,152 │
│ (Sequential)                    │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/downsampling_1      │ ?                         │     74,208 │
│ (Sequential)                    │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/downsampling_2      │ ?                         │    148,320 │
│ (Sequential)                    │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/stage_0             │ ?                         │  1,158,624 │
│ (Sequential)                    │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/stage_1             │ ?                         │  1,218,048 │
│ (Sequential)                    │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/stage_2             │ ?                         │ 10,907,712 │
│ (Sequential)                    │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/stage_3             │ ?                         │ 12,197,568 │
│ (Sequential)                    │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/final_pool          │ ?                         │          0 │
│ (GlobalAveragePooling2D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/final_layer_norm    │ ?                         │      1,536 │
│ (LayerNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ fourier_cnn/classification_head │ ?                         │    769,000 │
│ (Dense)                         │                           │            │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 26,516,968 (809.23 MB)

 Trainable params: 26,516,968 (809.23 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
from keras_core import utils

In [5]:
utils.plot_model(model)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [6]:
test = FourierConv2D(use_bias=True, activation='linear', data_format='channels_last')
model = models.Sequential([test])

In [7]:
ds = tfds.load('cifar10', split='train')

In [8]:
def gauss_kernel(channels, kernel_size, sigma):
    ax = tf.range(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0)
    xx, yy = tf.meshgrid(ax, ax)
    kernel = tf.exp(-(xx ** 2 + yy ** 2) / (2.0 * sigma ** 2))
    kernel = kernel / tf.reduce_sum(kernel)
    kernel = tf.tile(kernel[..., tf.newaxis], [1, 1, channels])
    return kernel

def gaussian_blur(img, kernel):
    gaussian_kernel = kernel
    gaussian_kernel = gaussian_kernel[..., tf.newaxis]

    return tf.nn.depthwise_conv2d(img, gaussian_kernel, [1, 1, 1, 1],
                                  padding='SAME', data_format='NHWC')

In [9]:
kernel = gauss_kernel(3, 3, 2)
def transform_cifar(x):
    x = tf.cast(x['image'], tf.float32) / 255
    y = gaussian_blur(x, kernel)
    return x, y

In [10]:
model.compile(loss='mse', optimizer='adam')
model.fit(ds.batch(64).map(transform_cifar))

2023-10-06 15:32:18.891804: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-10-06 15:32:18.892156: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-10-06 15:32:19.457668: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x1f487620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-06 15:32:19.457701: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1080,

 45/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3042        

2023-10-06 15:32:19.859795: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.1525


In [11]:
out = model.evaluate(ds.batch(64).map(transform_cifar).take(1))

2023-10-06 15:32:23.786450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-10-06 15:32:23.787011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - loss: 0.0141


In [12]:
plt.imshow(out[0])

TypeError: 'float' object is not subscriptable